In [ ]:
import os

In [ ]:
BUCKET = 'gs://edml'
os.environ['BUCKET'] = BUCKET

In [ ]:
!python --version

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

## Local training / Python

In [ ]:
%%bash
export PYTHONPATH=${PYTHONPATH}:~/event-driven-ml/edml-trainer
echo $PYTHONPATH

In [ ]:
%%bash
echo "bucket=${BUCKET}"
rm -rf model_trained_test
export PYTHONPATH=${PYTHONPATH}:~/event-driven-ml/edml-trainer
python3 -m trainer.task \
  --bucket=${BUCKET} \
  --output-dir=model_trained_test \
  --job-dir=./tmp \
  --train-steps=1000 \
  --nembeds 10 \
  --nnsize 10 5 \
  --eval-steps=2

## Local prediction

In [ ]:
%%writefile inputs.json
{"uuid": "b1", "dayofweek": 6, "hourofday": 7, "weekofyear": 48, "pickup_zone_name": "World Trade Center", "dropoff_zone_name": "Newark Airport", "passenger_count": 1, "distance": 18000.0}
{"uuid": "g1", "dayofweek": 3, "hourofday": 23, "weekofyear": 27, "pickup_zone_name": "World Trade Center", "dropoff_zone_name": "Times Sq/Theatre District", "passenger_count": 1, "distance": 3400.0}

I had this [issue](https://github.com/GoogleCloudPlatform/cloudml-samples/issues/415). Following instructions from [stackoverflow](https://stackoverflow.com/questions/48824381/gcloud-ml-engine-local-predict-runtimeerror-bad-magic-number-in-pyc-file)

First run command `gcloud ai-platform local predict` with `--verbosity debug` to find the correct path, then delete `*.pyc` files thes launch local train again.

```
%%bash
sudo find /usr/lib/google-cloud-sdk/lib/googlecloudsdk/command_lib/ml_engine -name '*.pyc' -delete
```

In [ ]:
%%bash
MODEL_LOCATION=$(ls -d $(pwd)/model_trained_test/export/exporter/* | tail -1)
echo $MODEL_LOCATION
gcloud ai-platform local predict --model-dir=$MODEL_LOCATION --json-instances=inputs.json

## Local training / Gcloud

In [ ]:
%%bash
gcloud ai-platform --help

In [ ]:
%%bash
echo $PATH

In [ ]:
#    --package-path=~/event-driven-ml/edml-trainer \

In [ ]:
%%bash
rm -rf model_trained_test
export PYTHONPATH=${PYTHONPATH}:~/event-driven-ml/edml-trainer:/usr/bin/python
gcloud ai-platform local train \
   --module-name=trainer.task \
   --package-path=../../edml-trainer/ \
   --job-dir=./tmp \
   -- \
   --train-steps=1000 \
   --output-dir=model_trained_test \
   --eval-steps=1

In [ ]:
%%bash
gsutil cp -r ~/event-driven-ml/edml-trainer/* gs://edml/ai-platform/edml-trainer/